In [2]:
import arff
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical


In [3]:
file = open("Dataset/final-dataset.arff")

In [4]:
def scrape_data():
    decoder = arff.ArffDecoder()
    data = decoder.decode(file, encode_nominal=True)

    vals = [val[0: -1] for val in data['data']]
    labels = [label[-1] for label in data['data']]

    for val in labels:
        if labels[val] != 0:
            labels[val] = 1
    
    training_data = vals[0: int(.9 * len(vals))]
    training_lables = labels[0: int(0.9 * len(vals))]
    validation_data = vals[int(.9 * len(vals)):]
    validation_labels = labels[int(.9 * len(vals)):]

    training_lables = to_categorical(training_lables, 5)
    validation_labels = to_categorical(validation_labels, 5)

    return np.asarray(training_data), np.asarray(training_lables), np.asarray(validation_data), np.asarray(validation_labels)

In [5]:
def generate_model(shape):
    model = Sequential()
    
    model.add(Dense(30, input_dim=shape,
                kernel_initializer='uniform', activation='relu'))

    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(5, activation='softmax'))
    print(model.summary())

    return model

In [6]:
data_train, label_train, data_eval, label_eval = scrape_data()

model = generate_model(len(data_train[0]))
model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                840       
                                                                 
 dropout (Dropout)           (None, 30)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 64)                7

In [8]:
tensorboard = TensorBoard(
    log_dir='logs/', histogram_freq=0, write_graph=True, write_images=True)

history = model.fit(data_train, label_train, validation_data=(
    data_eval, label_eval), epochs=2, callbacks=[tensorboard])

loss_history = history.history["loss"]

# evaluate the model's performance
print(model.evaluate(data_eval, label_eval))
print(model.evaluate(data_train, label_train))

Epoch 1/2
60769/60769 [==============================] - 193s 3ms/step - loss: 66.5870 - accuracy: 0.8945 - val_loss: 0.3793 - val_accuracy: 0.8956
Epoch 2/2
6753/6753 [==============================] - 11s 2ms/step - loss: 0.3793 - accuracy: 0.8956
[0.37926915287971497, 0.8956295847892761]
60769/60769 [==============================] - 98s 2ms/step - loss: 0.3795 - accuracy: 0.8960
[0.37945234775543213, 0.8960413932800293]
